In [ ]:
import pandas as pd
import numpy as np 
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df= pd.read_csv('bbc_text_cls.csv')

In [ ]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [ ]:
df[df.labels == 'business'].tail()

,text,labels
505,Trial begins of Spain's top banker\n\nThe tria...,business
506,UK economy ends year with spurt\n\nThe UK econ...,business
507,HealthSouth ex-boss goes on trial\n\nThe forme...,business
508,Euro firms miss out on optimism\n\nMore than 9...,business
509,Lacroix label bought by US firm\n\nLuxury good...,business


In [ ]:
df[df.labels == 'business']['text'].sample().shape

(1,)

In [ ]:
doc = df[df.labels == 'business']['text'].sample(random_state =42)

In [ ]:
def wrap(x):
  return textwrap.fill(x, replace_whitespace= False,fix_sentence_endings = True)

In [ ]:
doc.iloc[0]

'Christmas sales worst since 1981\n\nUK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.\n\nRetail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said. The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%. A number of retailers have already reported poor figures for December. Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.\n\nThe last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.\n\nThe ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures. Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003. The N

In [ ]:
print(wrap(doc.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [ ]:
sents = nltk.sent_tokenize(doc.iloc[0].split("\n",1)[1])
# notice the above lines that the ilox[0] return the document as a string 
# we split it in to two parts
# this is done as we want to seperate the title from the main text
# the first line is title 
# tyhen in the [title,text] we are indexin forthe text by the "[1] " at the at the end 


In [ ]:
doc.iloc[0].split("\n",1)

['Goodrem wins top female MTV prize',
 "\nPop singer Delta Goodrem has scooped one of the top individual prizes at the first Australian MTV Music Awards.\n\nThe 21-year-old singer won the award for best female artist, with Australian Idol runner-up Shannon Noll taking the title of best male at the ceremony. Goodrem, known in both Britain and Australia for her role as Nina Tucker in TV soap Neighbours, also performed a duet with boyfriend Brian McFadden. Other winners included Green Day, voted best group, and the Black Eyed Peas. Goodrem, Green Day and the Black Eyed Peas took home two awards each. As well as best female, Goodrem also took home the Pepsi Viewers Choice Award, whilst Green Day bagged the prize for best rock video for American Idiot. The Black Eyed Peas won awards for best R 'n' B video and sexiest video, both for Hey Mama. Local singer and songwriter Missy Higgins took the title of breakthrough artist of the year, with Australian Idol winner Guy Sebastian taking the hono

In [ ]:
sents

['\nUK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.',
 'Retail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said.',
 'The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%.',
 'A number of retailers have already reported poor figures for December.',
 'Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.',
 'The last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.',
 'The ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures.',
 'Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003.',
 'The November-De

In [ ]:
V = TfidfVectorizer(
    stop_words = stopwords.words('english'),
    norm='l1',
)
# important
# the norm ='l1' is applied to give equal importance to all th e  sentences as ther are sentences 
# of varying lenght in our input 

In [ ]:
X = V.fit_transform(sents)

In [ ]:
def get_sentence_score(tfidf_row):
  # return the mean of the the tfidf values of that row
  # the mean f only those values whivh are not "0"
  x = tfidf_row[tfidf_row != 0]
  return x.mean()
  

In [ ]:
scores = np.zeros(len(sents))
for i in range(len(sents)):
  score = get_sentence_score(X[i,:])
  scores[i] = score

In [ ]:
# we neend to sort the scores\
sort_idx =np.argsort(-scores)
# by default the argesort  sorts in ascending order thsu we need ro put the neagtive sighn

In [ ]:
print("this is the grnerated summary")
for i in sort_idx[:5]:
  print(wrap("%0.2f : %s" % (scores[i],sents[i])))

this is the grnerated summary
0.14 : A number of retailers have already reported poor figures for
December.
0.13 : However, reports from some High Street retailers highlight the
weakness of the sector.
0.12 : The ONS revised the annual 2004 rate of growth down from the
5.9% estimated in November to 3.2%.
0.10 : "Our view is the Bank of England will keep its powder dry and
wait to see the big picture."
0.10 : And a British Retail Consortium survey found that Christmas
2004 was the worst for 10 years.


In [ ]:
# there is one wat to chech if this is a good summary or not 
# it is to compare  it with its  title 
doc.iloc[0].split("\n",1)[0]

'Christmas sales worst since 1981'

In [ ]:
def summarise(text):
  sents = nltk.sent_tokenize(text)
  X = V.fit_transform(sents)

  scores = np.zeros(len(sents))
  for i in range(len(sents)):
    score = get_sentence_score(X[i,:])
    scores[i] = score
  sort_idx =np.argsort(-scores)


  for i in sort_idx[:5]:
    print(wrap("%0.2f : %s" % (scores[i],sents[i])))






In [ ]:
# thus above we group together everything that wwe have implemented so far
# we will now test t out 


In [ ]:
doc = df[df.labels == 'entertainment']['text'].sample(random_state=123)

summarise(doc.iloc[0].split("\n",1)[1])

0.11 : The Black Eyed Peas won awards for best R 'n' B video and
sexiest video, both for Hey Mama.
0.10 : The ceremony was held at the Luna Park fairground in Sydney
Harbour and was hosted by the Osbourne family.
0.10 : Goodrem, Green Day and the Black Eyed Peas took home two awards
each.
0.10 : Other winners included Green Day, voted best group, and the
Black Eyed Peas.
0.10 : The VH1 First Music Award went to Cher honouring her
achievements within the music industry.


In [ ]:
# the title is 

doc.iloc[0].split("\n",1)[0]

'Goodrem wins top female MTV prize'

In [ ]:
print(wrap(doc.iloc[0]))

Goodrem wins top female MTV prize

Pop singer Delta Goodrem has
scooped one of the top individual prizes at the first Australian MTV
Music Awards.

The 21-year-old singer won the award for best female
artist, with Australian Idol runner-up Shannon Noll taking the title
of best male at the ceremony.  Goodrem, known in both Britain and
Australia for her role as Nina Tucker in TV soap Neighbours, also
performed a duet with boyfriend Brian McFadden.  Other winners
included Green Day, voted best group, and the Black Eyed Peas.
Goodrem, Green Day and the Black Eyed Peas took home two awards each.
As well as best female, Goodrem also took home the Pepsi Viewers
Choice Award, whilst Green Day bagged the prize for best rock video
for American Idiot.  The Black Eyed Peas won awards for best R 'n' B
video and sexiest video, both for Hey Mama.  Local singer and
songwriter Missy Higgins took the title of breakthrough artist of the
year, with Australian Idol winner Guy Sebastian taking the honours f